# Data Processing

## Bus Data

### Downloading the Bus Data

This will download about 10GB of data, and may therefore take a while.

In [ ]:
import downloader

downloader.download_bus_data()

### Downloading the Bus Schedule Data

Compressed bus schedule data conforming to the GTFS Static standard can be found at the following:
- https://mobilitydatabase.org/feeds/gtfs/mdb-50
- https://transitfeeds.com/p/sfmta/60

The schedule currently in use can be found at: 
- https://www.sfmta.com/reports/gtfs-transit-data

To add a schedule to the project:
1. Download the static GTFS zip file.
2. Decompress the zip file as a directory.
3. Name the directory according to the start and end dates in the `calendar.txt` file within (i.e. if the dates are `20250315` and `20250509`, name the directory `2025-03-15_2025-05-09`).
4. Place the directory within `data/bus-static/sf/`.

We used the following schedules: 
- `2024-01-20_2024-03-01`
- `2024-03-02_2024-05-24`
- `2024-06-08_2024-08-16`
- `2024-06-22_2024-08-16`
- `2025-03-15_2025-05-09`

### Processing the Data
This will take a while, dependending on your computer's processing power. Expect it to take about 1 minute per processed day (a few hours total).

In [ ]:
import bus_processing

bus_processing.process_all()

## Calendar Data

In [ ]:
from IPython.display import display
import calendar_processing

calendar_df = calendar_processing.build_calendar()
display(calendar_df)

,holiday,monday,tuesday,wednesday,thursday,friday,saturday,sunday
date,,,,,,,,
2023-01-01,1,0,0,0,0,0,0,1
2023-01-02,1,1,0,0,0,0,0,0
2023-01-03,0,0,1,0,0,0,0,0
2023-01-04,0,0,0,1,0,0,0,0
2023-01-05,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
2025-04-10,0,0,0,0,1,0,0,0
2025-04-11,0,0,0,0,0,1,0,0
2025-04-12,0,0,0,0,0,0,1,0


## Weather Data